In [56]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os.path
from sklearn.model_selection import train_test_split


Create 2 layered NN

In [57]:
def create_model(inputs, weights, biases):
    #Create 2-layered NN
    input_layer= tf.matmul(inputs,weights['input'])
    # Hidden layer 1 (relu activation)
    hidden_layer1= tf.nn.relu(input_layer + biases['input'])
    #  Hidden layer 2 (relu activation)
    hidden_layer2= tf.nn.relu(tf.matmul(hidden_layer1,weights['hidden1'])+biases['hidden1'])
    # Output layer (linear activation)
    output_layer= tf.matmul(hidden_layer2, weights['hidden2']) + biases['hidden2']

    return output_layer


In [ ]:
#First Column is the label and others are the features
COLUMNS = ['className','handicapped-infants','water-project-cost-sharing','adoption-of-the-budget-resolution',
'physician-fee-freeze','el-salvador-aid','religious-groups-in-schools','anti-satellite-test-ban',
'aid-to-nicaraguan-contras','mx-missle','immigration','synfuels-corporation-cutback','education-spending',
'superfund-right-to-sue','crime','duty-free-exports','export-administration-act-south-africa']

# Read data into CSV file and convert ? to NaN
df = pd.read_csv(os.path.join("./house-votes-84.data.txt")
                       , names=COLUMNS
                       , skipinitialspace=True
                       , na_values="?")

Dea

In [58]:
#Rows with NaN values are removed from data
df.replace(["NaN"], np.nan, inplace = True)
df = df.dropna()
df = df.reset_index(drop=True)
  
#New row numbers after removing NaN values
rows_number=df.shape[0]

In [59]:
#Label column republican/democrat to 1 and 0
for i in range(0,len(COLUMNS)):
    cleanup_nums = {COLUMNS[i]: {"y":1, "n":0, "republican":1,"democrat":0}}
    df.replace(cleanup_nums, inplace=True)

One hot encoding of labels

In [60]:
#Make a 2 column matrix for the labels column
labels=np.zeros((rows_number, 2))
label=df['className']
print(label)
for i in range(0,len(label)):
    if(label[i]==0):
        labels[i,0]=0
        labels[i,1]=1
    else:
        labels[i,0]=1
        labels[i,1]=0
print (labels)
inputs=df[COLUMNS[2:]]
inputs=inputs.values

0      0
1      1
2      0
3      0
4      0
5      0
6      0
7      1
8      0
9      1
10     0
11     1
12     0
13     1
14     1
15     1
16     0
17     0
18     0
19     0
20     0
21     0
22     1
23     1
24     1
25     1
26     1
27     1
28     0
29     1
      ..
202    1
203    0
204    0
205    0
206    0
207    1
208    0
209    0
210    1
211    1
212    1
213    0
214    0
215    1
216    1
217    0
218    1
219    0
220    1
221    0
222    0
223    0
224    1
225    0
226    0
227    0
228    0
229    1
230    1
231    0
Name: className, Length: 232, dtype: int64
[[0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0

In [61]:
x_train, x_test, y_train, y_test = train_test_split(inputs, labels)


In [62]:
#HyperParameters Training
learning_rate = 0.5
epochs = 50
batch_size = 50 #small dataset

n_features = 15
n_calsses=2

#HyperParameters NN
hidden_nodes_layer1=10
hidden_nodes_layer2=10


In [63]:
#Define variables for weights and biases
input_weights = tf.Variable(tf.random_normal([n_features, hidden_nodes_layer1]))
input_biases = tf.Variable(tf.zeros([1,hidden_nodes_layer1]))

hidden1_weights = tf.Variable(tf.random_normal([hidden_nodes_layer1, hidden_nodes_layer2]))
hidden1_biases = tf.Variable(tf.zeros([1,hidden_nodes_layer2]))

hidden2_weights = tf.Variable(tf.random_normal([hidden_nodes_layer2, n_calsses]))
hidden2_biases = tf.Variable(tf.zeros([1,n_calsses]))

weights={'input':input_weights,'hidden1':hidden1_weights,'hidden2':hidden2_weights}
biases={'input':input_biases,'hidden1':hidden1_biases,'hidden2':hidden2_biases}

In [64]:
#Placeholders to initialize 
x = tf.placeholder(tf.float32, [None, n_features])
y = tf.placeholder(tf.float32, [None, n_calsses])

#Create the NN model
prediction=create_model(x,weights,biases)


In [65]:
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_function)

correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


In [66]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

with tf.Session() as sess:
    sess.run(init)    
    #Training cycle
    # Loop epochs
    for epoch in range(epochs):
        avg_cost = 0       
        total_batch = int(len(x_train)/batch_size)
        X_batches = np.array_split(x_train, total_batch)
        Y_batches = np.array_split(y_train, total_batch)
        # Loop batches
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            # Run optimization and loss 
            _, cost = sess.run([optimizer, loss_function], feed_dict={x: batch_x,
                                                          y: batch_y})
            # average cost
            avg_cost += cost / total_batch
        print("Epoch:", epoch+1, "cost=", avg_cost)
    
    #accuracy
    print("Accuracy: " + str(accuracy.eval(feed_dict={x: x_test,
                                                          y: y_test})))



('Epoch:', 1, 'cost=', 27.517919858296715)
('Epoch:', 2, 'cost=', 0.7076063950856526)
('Epoch:', 3, 'cost=', 0.7023968696594238)
('Epoch:', 4, 'cost=', 0.6983156204223633)
('Epoch:', 5, 'cost=', 0.6952630877494812)
('Epoch:', 6, 'cost=', 0.6930194695790608)
('Epoch:', 7, 'cost=', 0.6908019781112671)
('Epoch:', 8, 'cost=', 0.6885601878166199)
('Epoch:', 9, 'cost=', 0.6862834294637045)
('Epoch:', 10, 'cost=', 0.6840344468752544)
('Epoch:', 11, 'cost=', 0.681976060072581)
('Epoch:', 12, 'cost=', 0.6787038048108419)
('Epoch:', 13, 'cost=', 0.6744384169578552)
('Epoch:', 14, 'cost=', 0.6699090202649435)
('Epoch:', 15, 'cost=', 0.6668328245480855)
('Epoch:', 16, 'cost=', 0.655076801776886)
('Epoch:', 17, 'cost=', 0.6395849784215292)
('Epoch:', 18, 'cost=', 0.6129528482755026)
('Epoch:', 19, 'cost=', 0.5619836052258809)
('Epoch:', 20, 'cost=', 0.4810996552308401)
('Epoch:', 21, 'cost=', 0.4090161124865214)
('Epoch:', 22, 'cost=', 0.3544098138809204)
('Epoch:', 23, 'cost=', 0.3135599692662557)